# cRT (T-F)

In [1]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

import tensorflow as tf
import tensorflow_addons as tfa
from keras import layers

import librosa
from audiomentations import SpecCompose, SpecFrequencyMask

from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import pickle
import argparse
import wandb
from wandb.keras import WandbCallback
wandb.init(project="DACON_235910", name="cRT")

parser = argparse.ArgumentParser(description="cRT")
parser.add_argument('--feature', default="melspec", type=str) # mfcc or melspec
parser.add_argument('--pretrained_model', default="efficientnetb0", type=str)
parser.add_argument('--resize_size', default=224, type=int)
parser.add_argument('--sampling_rate', default=16000, type=int)
# parser.add_argument('--frequency_mask', default=0, type=float)
parser.add_argument('--optimizer', default="sgd", type=str) # sgd or adam
parser.add_argument('--loss', default="bc", type=str) # bc or fl
parser.add_argument('--learning_rate', default=0.001, type=float)
parser.add_argument('--batch_size', default=32, type=int)
parser.add_argument('--epochs', default=100, type=int)
parser.add_argument('--cv', default=10, type=int)
parser.add_argument('--seed', default=1011, type=int)
args = parser.parse_args('')

wandb.config.update(args)

feature = args.feature
pretrained_model = args.pretrained_model
resize_size = args.resize_size
sampling_rate = args.sampling_rate
# frequency_mask = args.frequency_mask
optimizer = args.optimizer
loss = args.loss
learning_rate = args.learning_rate
BATCH_SIZE = args.batch_size
EPOCHS = args.epochs
cv = args.cv
seed = args.seed

def set_seeds(seed=seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

# augment = SpecCompose([SpecFrequencyMask(p=frequency_mask)])
    
with open('data/train_df.pkl', 'rb') as f:
    train_df = pickle.load(f)
with open('data/test_df.pkl', 'rb') as f:
    test_df = pickle.load(f)
    
train_df.head()

wandb: Currently logged in as: gnoeyheat (use `wandb login --relogin` to force relogin)


,data,id,age,gender,respiratory_condition,fever_or_muscle_pain,covid19
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,24,female,0,1,0
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",2,51,male,0,0,0
2,"[2.7372453e-09, -1.0615647e-08, 5.2142607e-08,...",3,22,male,0,0,0
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",4,29,female,1,0,0
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",5,23,male,0,0,0


In [2]:
train_df["covid19"].value_counts(normalize=True)

0    0.91958
1    0.08042
Name: covid19, dtype: float64

## Data Generation

In [3]:
# train_df = pd.read_csv("data/train_data.csv")
# test_df = pd.read_csv("data/test_data.csv")

# train_folder = "data/train/"
# test_folder = "data/test/"

# def dataset(folder, df):
#     dataset = []
#     for uid in tqdm(df['id']):
#         path = os.path.join(folder, str(uid).zfill(5)+'.wav')
#         y, sr = librosa.load(path, sr=sampling_rate)
#         y = librosa.util.normalize(y)
#         dataset.append([y])
#     dataset = pd.DataFrame(dataset, columns=['data'])
#     dataset = pd.concat([dataset, df], axis=1)
#     return dataset

# train_df = dataset(train_folder, train_df)
# test_df = dataset(test_folder, test_df)

# with open('train_df.pkl', 'wb') as f:
#     pickle.dump(train_df, f, pickle.HIGHEST_PROTOCOL)
# with open('test_df.pkl', 'wb') as f:
#     pickle.dump(test_df, f, pickle.HIGHEST_PROTOCOL)

## Preprocessing

In [4]:
def preprocess_dataset(data):

    frame_length = 0.025
    frame_stride = 0.010

    input_nfft = int(round(sampling_rate*frame_length))
    input_stride = int(round(sampling_rate*frame_stride))

    extracted_features = []
    for i in tqdm(data):
        temp_S = []
        for nfft, stride in zip([input_nfft, input_nfft*4, input_nfft],
                                [input_stride, input_stride, input_stride*4]):
            if feature == "mfcc":
                S = librosa.feature.mfcc(y=i,
                                         sr=sampling_rate,
                                         n_mfcc=40,
                                         n_fft=nfft,
                                         hop_length=stride)
            elif feature == "melspec":
                S = librosa.feature.melspectrogram(y=i,
                                                   sr=sampling_rate,
                                                   n_mels=128,
                                                   n_fft=nfft,
                                                   hop_length=stride)
                S = librosa.power_to_db(S, ref=np.max)
            S = tf.image.resize(S[:, :, np.newaxis], (resize_size, resize_size))
            temp_S.append(S[:, :, 0])
        S = np.stack(temp_S, axis=2)
        extracted_features.append(S)
    return extracted_features

X = np.array(preprocess_dataset(train_df["data"]))
X_test = np.array(preprocess_dataset(test_df["data"]))
y = train_df["covid19"]

X.shape, y.shape, X_test.shape

100%|██████████████████████████████████████████████████████████████████████████████| 5732/5732 [05:44<00:00, 16.64it/s]


((3805, 224, 224, 3), (3805,), (5732, 224, 224, 3))

In [5]:
def preprocess_feature(df):
    temp = df.copy()
    temp["condition1"] = temp["respiratory_condition"] + temp["fever_or_muscle_pain"]
    temp["condition2"] = temp["respiratory_condition"] * temp["fever_or_muscle_pain"]
    temp = temp.drop(["id", "age", "gender", "respiratory_condition", "fever_or_muscle_pain"], axis=1)
    return temp

train_df = preprocess_feature(train_df)
test_df = preprocess_feature(test_df)

X_test_tab = test_df[["condition1", "condition2"]].values

X_test_tab.shape

(5732, 2)

## Training

In [6]:
validations = []
thresholds = []
predictions = []

idx=0

skf = StratifiedKFold(n_splits=cv)
for train_index, val_index in tqdm(skf.split(X, y)):
    
    idx+=1

    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    X_train_tab = train_df[["condition1", "condition2"]].values[train_index]
    X_val_tab = train_df[["condition1", "condition2"]].values[val_index]
    
    train_ds = (
        tf.data.Dataset.from_tensor_slices(((X_train, X_train_tab), y_train))
        .shuffle(len(X_train))
        .batch(BATCH_SIZE)
        .prefetch(tf.data.experimental.AUTOTUNE)
    )

    val_ds = (
        tf.data.Dataset.from_tensor_slices(((X_val, X_val_tab), y_val))
        .batch(BATCH_SIZE)
        .prefetch(tf.data.experimental.AUTOTUNE)
    )

    if pretrained_model == "efficientnetb0":
        encoder = tf.keras.applications.EfficientNetB0(
            include_top=False,
            weights="imagenet",
            pooling='avg',
        )

    inp = tf.keras.Input(shape=(resize_size, resize_size, 3))
    tab = tf.keras.Input(shape=(X_test_tab.shape[1],))
    x = encoder(inp)
    x = layers.Concatenate()([x, tab])
    oup = layers.Dense(1, activation="sigmoid")(x)
    model = tf.keras.Model(inputs=[inp, tab], outputs=oup)

    lr = tf.keras.optimizers.schedules.CosineDecay(learning_rate, decay_steps=1000)
    if optimizer == "adam":
        optim = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == "sgd":
        optim = tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9)
        
    label_smoothing=0
    loss_function = tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing)

    model.compile(
        optimizer=optim,
        loss=loss_function,
    )
    
    checkpoint_filepath=f"load_model/{parser.description}_{idx}_1stage"

    checkpoint_callback = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True,
        ),
        tf.keras.callbacks.ModelCheckpoint(
            checkpoint_filepath,
            monitor="val_loss",
            save_best_only=True,
            save_weights_only=True
        )
    ]

    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS,
        callbacks=[checkpoint_callback, WandbCallback()],
    )
    
    X_train = np.concatenate((X_train[np.where(y_train==0, True, False)][:y_train.sum()],
                              X_train[np.where(y_train==1, True, False)]))
    X_train_tab = np.concatenate((X_train_tab[np.where(y_train==0, True, False)][:y_train.sum()],
                                  X_train_tab[np.where(y_train==1, True, False)]))
    y_train = np.concatenate((y_train[np.where(y_train==0, True, False)][:y_train.sum()],
                              y_train[np.where(y_train==1, True, False)]))

    train_ds = (
        tf.data.Dataset.from_tensor_slices(((X_train, X_train_tab), y_train))
        .shuffle(len(X_train))
        .batch(BATCH_SIZE)
        .prefetch(tf.data.experimental.AUTOTUNE)
    )
    
    encoder.trainable = False
    
    lr = tf.keras.optimizers.schedules.CosineDecay(learning_rate*0.1, decay_steps=1000)
    if optimizer == "adam":
        optim = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer == "sgd":
        optim = tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9)

    model.compile(
        optimizer=optim,
        loss=loss_function,
    )
    
    checkpoint_filepath=f"load_model/{parser.description}_{idx}_2stage"

    checkpoint_callback = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True,
        ),
        tf.keras.callbacks.ModelCheckpoint(
            checkpoint_filepath,
            monitor="val_loss",
            save_best_only=True,
            save_weights_only=True
        )
    ]

    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS,
        callbacks=[checkpoint_callback, WandbCallback()],
    )

    max_f1 = 0
    threshold = 0
    for temp_threshold in np.linspace(0.05, 0.95, 19):
        temp_f1 = f1_score(y_val, np.where(model.predict(val_ds)>temp_threshold, 1, 0), average="macro")
        if temp_f1 > max_f1:
            max_f1 = temp_f1
            threshold = temp_threshold
    
    print(f"idx:{idx}, f1:{max_f1}, threshold:{threshold}")
    
    validations.append(max_f1)
    thresholds.append(threshold)
    predictions.append(model.predict([X_test, X_test_tab]))

val_f1 = np.mean(validations, axis=0)

print("validation_macro-f1: ", val_f1)
wandb.log({'validation_macro-f1': val_f1})

0it [00:00, ?it/s]

Epoch 1/100
107/107 [==============================] - 50s 257ms/step - loss: 0.3696 - val_loss: 0.8882 - _timestamp: 1655803284.0000 - _runtime: 683.0000
Epoch 2/100
107/107 [==============================] - 26s 239ms/step - loss: 0.2924 - val_loss: 0.5796 - _timestamp: 1655803312.0000 - _runtime: 711.0000
Epoch 3/100
107/107 [==============================] - 25s 238ms/step - loss: 0.2711 - val_loss: 0.4486 - _timestamp: 1655803339.0000 - _runtime: 738.0000
Epoch 4/100
107/107 [==============================] - 26s 239ms/step - loss: 0.2572 - val_loss: 0.3666 - _timestamp: 1655803365.0000 - _runtime: 764.0000
Epoch 5/100
107/107 [==============================] - 24s 224ms/step - loss: 0.2480 - val_loss: 0.4127 - _timestamp: 1655803392.0000 - _runtime: 791.0000
Epoch 6/100
107/107 [==============================] - 26s 240ms/step - loss: 0.2358 - val_loss: 0.2961 - _timestamp: 1655803418.0000 - _runtime: 817.0000
Epoch 7/100
107/107 [==============================] - 24s 224ms/step 

1it [08:30, 510.49s/it]

Epoch 1/100
107/107 [==============================] - 36s 256ms/step - loss: 0.3829 - val_loss: 0.6040 - _timestamp: 1655803796.0000 - _runtime: 1195.0000
Epoch 2/100
107/107 [==============================] - 25s 237ms/step - loss: 0.2918 - val_loss: 0.3535 - _timestamp: 1655803823.0000 - _runtime: 1222.0000
Epoch 3/100
107/107 [==============================] - 24s 227ms/step - loss: 0.2720 - val_loss: 0.3996 - _timestamp: 1655803848.0000 - _runtime: 1247.0000
Epoch 4/100
107/107 [==============================] - 24s 229ms/step - loss: 0.2605 - val_loss: 0.3027 - _timestamp: 1655803874.0000 - _runtime: 1273.0000
Epoch 5/100
107/107 [==============================] - 24s 224ms/step - loss: 0.2501 - val_loss: 0.3128 - _timestamp: 1655803900.0000 - _runtime: 1299.0000
Epoch 6/100
107/107 [==============================] - 25s 235ms/step - loss: 0.2425 - val_loss: 0.2779 - _timestamp: 1655803925.0000 - _runtime: 1324.0000
Epoch 7/100
107/107 [==============================] - 24s 223ms

2it [18:19, 556.79s/it]

...

Epoch 1/100
108/108 [==============================] - 35s 249ms/step - loss: 0.3794 - val_loss: 0.5337 - _timestamp: 1655807460.0000 - _runtime: 4859.0000
Epoch 2/100
108/108 [==============================] - 26s 237ms/step - loss: 0.2951 - val_loss: 0.3981 - _timestamp: 1655807487.0000 - _runtime: 4886.0000
Epoch 3/100
108/108 [==============================] - 25s 235ms/step - loss: 0.2758 - val_loss: 0.2831 - _timestamp: 1655807514.0000 - _runtime: 4913.0000
Epoch 4/100
108/108 [==============================] - 25s 228ms/step - loss: 0.2609 - val_loss: 0.3217 - _timestamp: 1655807540.0000 - _runtime: 4939.0000
Epoch 5/100
108/108 [==============================] - 25s 229ms/step - loss: 0.2548 - val_loss: 0.3006 - _timestamp: 1655807566.0000 - _runtime: 4965.0000
Epoch 6/100
108/108 [==============================] - 25s 228ms/step - loss: 0.2413 - val_loss: 0.3029 - _timestamp: 1655807592.0000 - _runtime: 4991.0000
Epoch 7/100
108/108 [==============================] - 25s 228ms

9it [1:16:48, 482.18s/it]

Epoch 1/100
108/108 [==============================] - 35s 247ms/step - loss: 0.3795 - val_loss: 0.7201 - _timestamp: 1655807879.0000 - _runtime: 5278.0000
Epoch 2/100
108/108 [==============================] - 25s 233ms/step - loss: 0.2904 - val_loss: 0.4027 - _timestamp: 1655807905.0000 - _runtime: 5304.0000
Epoch 3/100
108/108 [==============================] - 25s 232ms/step - loss: 0.2776 - val_loss: 0.3092 - _timestamp: 1655807932.0000 - _runtime: 5331.0000
Epoch 4/100
108/108 [==============================] - 24s 221ms/step - loss: 0.2613 - val_loss: 0.3239 - _timestamp: 1655807957.0000 - _runtime: 5356.0000
Epoch 5/100
108/108 [==============================] - 24s 221ms/step - loss: 0.2493 - val_loss: 0.3277 - _timestamp: 1655807982.0000 - _runtime: 5381.0000
Epoch 6/100
108/108 [==============================] - 25s 232ms/step - loss: 0.2472 - val_loss: 0.2713 - _timestamp: 1655808009.0000 - _runtime: 5408.0000
Epoch 7/100
108/108 [==============================] - 24s 224ms

10it [1:25:00, 510.00s/it]

validation_macro-f1:  0.5503708840943518


## Inference

In [7]:
threshold = np.mean(thresholds)
test_df["covid19"] = np.where(np.mean(predictions, axis=0)>threshold, 1, 0)

submission = pd.read_csv('data/sample_submission.csv')
submission['covid19'] = test_df['covid19']
submission.to_csv('submission.csv', index=False)

test_df['covid19'].value_counts(normalize=True)

0    0.974878
1    0.025122
Name: covid19, dtype: float64